## Overview

This notebook is intended to:
- Demonstrate how to access time series data for meters and SCADAs using Awesense's Energy Data Model (EDM) SQL API.

Please refer to the [main_concepts.ipynb](../2_main_concepts/main_concepts.ipynb) notebook for a high-level introduction to and simpler examples of the core views and functions available in Awesense's Energy Data Model (EDM).

---

## Set up

In [10]:
import getpass
import plotly.express as px
import plotly.io as pio
import time
import urllib.parse
import pandas as pd

# Allow charts to persist between notebook sessions.
pio.renderers.default='notebook'

**Connection**

Enter the EDM server address and the login credentials provided by Awesense. If you do not have the credentials, or have any trouble connecting, please contact api@awesense.com.
<span style='color:red'> **Please do NOT store the credentials in the notebook, nor share them with anyone.** </span>

In [11]:
edm_address = getpass.getpass(prompt='EDM server address: ')

print('\nEDM login information')
edm_name = getpass.getpass(prompt='Username: ')
edm_password = getpass.getpass(prompt='Password: ')
edm_password = urllib.parse.quote(edm_password)

%load_ext sql
%sql postgresql://$edm_name:$edm_password@$edm_address/edm
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

# Delete the credential variables for security purpose.
del edm_name, edm_password

EDM server address: ········

EDM login information
Username: ········
Password: ········
The sql extension is already loaded. To reload it, use:
  %reload_ext sql


---

## Time Series
Various temporary views are created below to optimize query performance within the notebook.

### Meter Time Series

*High level approach*
- Create a temporary view `grid_element_metric`, containing information on meters for grid elements and convenient access to the actual data.
- The `grid_get_downstream()` function is used to gather information of meters that are downstream of a specified grid element.
- Time series are retrieved using the `ts_data_source_select()` function.

**Downstream of a Grid** 
- Specify `grid_id` and `grid_element_id` whose downstream meter data to fetch for.

In [12]:
grid_id = input('Grid Id: ') # awefice
grid_element_id = input('Grid Element Id: ') # line_segment_57

Grid Id: awefice
Grid Element Id: transformer_2


Check when this grid was last updated.

In [4]:
%%sql

SELECT last_updated
FROM grid
WHERE grid_id = '{grid_id}';

last_updated
2021-01-01 20:00:00+00:00


In [18]:
##Finding the first branching after the first element - grid_element_id

Tree = %sql SELECT grid_element_id, terminal1_cn, terminal2_cn FROM grid_get_downstream('{grid_id}', '{grid_element_id}');
Tree = Tree.DataFrame()

AA = Tree[Tree['terminal1_cn']==Tree[Tree['grid_element_id']==grid_element_id]['terminal2_cn'].iloc[0]]
if (AA['terminal1_cn'].value_counts()>1).iloc[0]:
    print(f"First branching includes {AA}")
else:
    while ((AA['terminal1_cn'].value_counts()==1).iloc[0]):
        AA = Tree[Tree['terminal1_cn']==AA['terminal2_cn'].iloc[0]]
        if (AA['terminal1_cn'].value_counts()>1).iloc[0]:
                print(f"First branching includes {AA}")
                break
                


First branching includes     grid_element_id terminal1_cn terminal2_cn
7   line_segment_11       con_21       con_24
9  line_segment_134       con_21      con_342


In [28]:
%%sql 
select * from grid_get_downstream('{grid_id}', '{grid_element_id}')
limit 10;

grid_id,grid_element_id,type,customer_type,phases,is_underground,is_producer,is_consumer,is_switchable,switch_is_open,terminal1_cn,terminal2_cn,power_flow_direction,upstream_grid_element_id,geometry,meta
awefice,batt_10,Battery,battery,ABC,None,True,True,False,None,con_135_t2,VCN.batt_10,both,None,0101000020E61000000B45A2539AC65EC0DDB431982CA24840,"{'name': 'batt_10', 'label': 'batt_10', 'sdp_id': None, 'latitude': None, 'meter_id': 'm_10', 'longitude': None, 'lv_feeder': None, 'mv_feeder': None, 'ownership': 'private', 'make_model': 'LG CHEM RESU10H', 'enclosure_id': None, 'voltage_level': 'LV', 'capacity_usable': 9.6, 'commission_date': None, 'capacity_nominal': 9.8, 'manufacture_date': None, 'efficiency_round_trip': 97.96, 'max_peak_charge_power': 7.0, 'parent_transformer_id': 'transformer_36', 'max_peak_discharge_power': 7.0, 'max_continuous_charge_power': 5.0, 'max_continuous_discharge_power': 5.0}"
awefice,batt_4,Battery,battery,C,None,True,True,False,None,con_81_t2,VCN.batt_4,both,None,0101000020E61000006478F578A6C65EC021220BDF3BA24840,"{'name': 'batt_4', 'label': 'batt_4', 'sdp_id': None, 'latitude': None, 'meter_id': 'm_4', 'longitude': None, 'lv_feeder': None, 'mv_feeder': None, 'ownership': 'private', 'make_model': 'Tesla Powerwall 2', 'enclosure_id': None, 'voltage_level': 'LV', 'capacity_usable': 13.2, 'commission_date': None, 'capacity_nominal': 13.5, 'manufacture_date': None, 'efficiency_round_trip': 97.78, 'max_peak_charge_power': 7.0, 'parent_transformer_id': 'transformer_26', 'max_peak_discharge_power': 7.0, 'max_continuous_charge_power': 5.0, 'max_continuous_discharge_power': 5.0}"
awefice,evch_10,EVCharger,ev_charger,ABC,None,True,True,False,None,con_135_t2,VCN.evch_10,both,None,0101000020E61000000B45A2539AC65EC0DDB431982CA24840,"{'name': None, 'label': 'evch_10', 'sdp_id': None, 'latitude': None, 'meter_id': 'm_10', 'longitude': None, 'lv_feeder': None, 'ownership': 'private', 'make_model': 'ChargePoint Home Flex [CPH50-NEMA14-50-L23]', 'rating_kva': '', 'max_current': 32, 'max_voltage': 240, 'is_dc_charger': False, 'voltage_level': 'LV', 'charging_level': 'level_2', 'connector_type': 'NEMA 14-30', 'active_charging_power': 7680, 'parent_transformer_id': 'transformer_36', 'active_generation_power': 0}"
awefice,evch_12,EVCharger,ev_charger,ABC,None,True,True,False,None,con_152_t2,VCN.evch_12,both,None,0101000020E610000032415CF294C65EC0F99D2EC423A24840,"{'name': None, 'label': 'evch_12', 'sdp_id': None, 'latitude': None, 'meter_id': 'm_12', 'longitude': None, 'lv_feeder': None, 'ownership': 'private', 'make_model': 'Tesla Supercharger [Tesla]', 'rating_kva': '', 'max_current': 330, 'max_voltage': 410, 'is_dc_charger': True, 'voltage_level': 'LV', 'charging_level': 'level_3', 'connector_type': 'TESLA', 'active_charging_power': 135000, 'parent_transformer_id': 'transformer_47', 'active_generation_power': 0}"
awefice,evch_3,EVCharger,ev_charger,A,None,True,True,False,None,con_40_t2,VCN.evch_3,both,None,0101000020E61000005BCDA4FDBEC65EC0E0B44C793CA24840,"{'name': None, 'label': 'evch_3', 'sdp_id': None, 'latitude': None, 'meter_id': 'm_3', 'longitude': None, 'lv_feeder': None, 'ownership': 'private', 'make_model': 'Tesla Gen 3 Wall Connector [1457768-**-*]', 'rating_kva': '', 'max_current': 16, 'max_voltage': 240, 'is_dc_charger': False, 'voltage_level': 'LV', 'charging_level': 'level_2', 'connector_type': 'TESLA', 'active_charging_power': 3840, 'parent_transformer_id': 'transformer_16', 'active_generation_power': 0}"
awefice,line_segment_1,ACLineSegment,line_segment,ABC,False,False,False,False,None,con_365,con_60,both,None,0102000020E610000002000000B4D96C77B5C65EC0C01F5CB538A248405E145099A2C65EC0F1905C1B38A24840,"{'name': '1', 'label': '1', 'system': 'delta', 'rating_a': 184, 'materials': 'Al_Steel', 'latitude_1': None, 'latitude_2': None, 'make_model': '#2 ACSR', 'line_length': 83.91, 'longitude_1': None, 'longitude_2': None, 'is_insulated': False, 'ph_ph_voltage': 12500, 'voltage_level': 'MV', 'wire_diameter':

In [39]:
#Retrieving elements, connections and phases from all elements downstream
Tree2 = %sql SELECT grid_element_id, terminal1_cn, terminal2_cn, phases FROM grid_get_downstream('{grid_id}', '{grid_element_id}');
Tree2 = Tree2.DataFrame()

#Merging elements that come one following the other
BB = Tree2.merge(Tree2, left_on='terminal2_cn', right_on='terminal1_cn',suffixes=('_upper', '_lower'))

#Tracing elements where the phases change from ABC to non-ABC
BB[(BB['phases_upper']=='ABC') & (BB['phases_lower']!='ABC')]

,grid_element_id_upper,terminal1_cn_upper,terminal2_cn_upper,phases_upper,grid_element_id_lower,terminal1_cn_lower,terminal2_cn_lower,phases_lower
1,line_segment_1,con_365,con_60,ABC,switchable_element_25,con_60,con_62,C
6,line_segment_12,con_344,con_346,ABC,switchable_element_15,con_346,con_32,A
9,line_segment_14,con_26,con_29,ABC,switchable_element_30,con_29,con_84,B
